In [1]:
from scipy.spatial import cKDTree
import pyfits, mechanize, bz2, os
from matplotlib import pyplot as pl
from numpy import *
from scipy.optimize import leastsq
import astropy.stats as sta
from scipy import ndimage
from glob import glob
import pandas as pd  #為了讀csv檔。csv檔是以逗點分割的檔案。
import requests
from bs4 import BeautifulSoup
from astropy.io import ascii
import concurrent.futures
from time import sleep
from astropy.table import Table

In [15]:
header = {
'Upgrade-Insecure-Requests': '1',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
'Referer': 'http://irsa.ipac.caltech.edu/ibe/docs/ptf/images/level1/',
'Cookie': '__utma=253438417.1413587276.1479455055.1486991316.1487038069.11; __utmz=253438417.1479459352.2.2.utmcsr=yahoo|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided); __utmc=253438417; usrkey="7820253d-5a48-472a-b8a4-44fa98bcc713/ncuiarex@astro.ncu.edu.tw"; JOSSO_SESSIONID=997565387D0A6064085E0F9D428BB981'
}    

In [3]:
#column_names = ['EPIC','HA663','HA656','HA663-HA656']  

#df = pd.DataFrame([ ], columns = column_names)
#df.to_csv('test3.csv', index=False)

In [3]:
def append_csv(x1, x2, x3, x4):

    df = pd.read_csv('ptf_ha_substraction_k2c1.csv')
    column_names = ['EPIC','HA663','HA656','HA663-HA656']  
    
    if x1 not in list(df['EPIC'])  :
    
    
        df2 = pd.DataFrame([[x1, x2, x3, x4]], columns = column_names)

        df_add = df.append(df2)

        df_add.to_csv('ptf_ha_substraction_k2c1.csv', index = False)
    #else:
        #print x3,'already exists'

In [2]:
os.getcwd()

'D:\\kepler_ptf_research\\k2_c1'

In [6]:
#os.chdir('..')

In [7]:
os.getcwd()

'D:\\python_PTF_data_k2c1'

In [5]:
dfs = pd.read_csv('K2_M_C1_ptf_h.csv')

In [6]:
#dfs['EPIC'][0:2]
ra_list = list(dfs['RA'][0:7])
dec_list = list(dfs['DEC'][0:7])
epic_list = list(dfs['EPIC'][0:7])

In [7]:
epic_list
ra_list

[174.69463500000001,
 175.25185099999999,
 175.08766799999998,
 176.619427,
 176.61401000000001,
 174.53988100000001,
 175.24243799999999]

In [8]:
# mulp. targers' files downloading test, success! (debuging is finished)

for s in range(len(ra_list)):
    input_ra_dec(ra_list[s], dec_list[s], epic_list[s])

NameError: name 'input_ra_dec' is not defined

In [5]:
sss1 = os.getcwd()
sss1

'/media/careyjohn/DATA/python_PTF_data_k2c1'

In [9]:
#Newest one!! 20170210

def input_ra_dec(ra, dec, epic):
    
    df = pd.read_csv('ptf_ha_substraction_k2c1.csv')
    column_names = ['EPIC','HA663','HA656','HA663-HA656']  
    
    if epic not in list(df['EPIC'])  :
    
        res = requests.get("http://irsa.ipac.caltech.edu/ibe/search/ptf/images/level1?POS={0},{1}".format(ra, dec), headers = header) 
        #print (res.text) #印出網頁內容
        soup = BeautifulSoup(res.text, 'lxml')
        data = ascii.read(soup.text)
        ha656_list = []
        ha663_list = []
        a = data['filter']
        af3 = data['afilename3']
        for i in range(0, len(af3)):
            if a[i] == "HA656":
                 ha656_list.append(af3[i])
            if a[i] == "HA663":
                 ha663_list.append(af3[i])


        url_ha663 = "http://irsa.ipac.caltech.edu/ibe/data/ptf/images/level1/"+ha663_list[0]
        url_ha656 = "http://irsa.ipac.caltech.edu/ibe/data/ptf/images/level1/"+ha656_list[0]
        ha663_filename = url_ha663.split('/')[-1]
        ha656_filename = url_ha656.split('/')[-1]
        # NOTE the stream=True parameter


        #os.chdir("D:/python_PTF_data_k2c1")  #for windows
        os.chdir("/media/careyjohn/DATA/python_PTF_data_k2c1")  #for Linux

        if not os.path.exists('ktwo'+str(epic)+'_C1'): # 若無該資料夾，則新建
            os.mkdir('ktwo'+str(epic)+'_C1')

        #os.mkdir('ktwo'+str(epic)+'_C1')

        #os.chdir("D:/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1")  #for windows
        os.chdir("/media/careyjohn/DATA/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1") # for Linux
        

        def P_ha656(str1):

            for i in range(len(str1)):
                if str1[i] == 'P':
                    return str1[i:]

        def P_ha663(str2):

            for i in range(len(str2)):
                if str2[i] == 'P':
                    return str2[i:]

        if not os.path.exists(P_ha663(ha663_list[0])) :

            r_ha663 = requests.get( url_ha663, stream=True, headers = header)
            with open(ha663_filename, 'wb') as f663:
                for chunk in r_ha663.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        f663.write(chunk)

        if not os.path.exists(P_ha656(ha656_list[0])):

            r_ha656 = requests.get( url_ha656, stream=True, headers = header)
            with open(ha656_filename, 'wb') as f656:
                for chunk in r_ha656.iter_content(chunk_size=1024): 
                    if chunk: # filter out keep-alive new chunks
                        f656.write(chunk)
            #return ha663_filename, ha656_filename

        ID = 'ktwo'+str(epic)+'_C1'  
        mra =   [ra]
        mdec = [dec]
        ptf_sig = 2.
        ptf_dm = 0.5
        #fn11 = 'PTF_201304293062_c_p_scie_t072054_u015955703_f11_p002958_c10.ctlg'
        #fn12 = 'PTF_201304293120_c_p_scie_t072914_u015957839_f12_p002958_c10.ctlg'

        #fn1 = '//media//careyjohn/OS//Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//'+fn11  #Linux 路徑
        #fn2 = '//media//careyjohn//OS//Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//'+fn12
        #fn1 = 'C://Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//'+fn11
        #fn2 = 'C://Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//'+fn12


        fn1 = P_ha656(ha656_list[0])
        fn2 = P_ha663(ha663_list[0])

        ptf1 = pyfits.open(fn1)
        ptf2 = pyfits.open(fn2)

        ra1 = ptf1[1].data['ALPHAWIN_J2000']
        dec1 = ptf1[1].data['DELTAWIN_J2000']
        mag1 = ptf1[1].data['MAG_AUTO']+ptf1[1].data['ZEROPOINT']

        ra2 = ptf2[1].data['ALPHAWIN_J2000']
        dec2 = ptf2[1].data['DELTAWIN_J2000']
        mag2 = ptf2[1].data['MAG_AUTO']+ptf2[1].data['ZEROPOINT']

        def crossmatch(X1, X2, max_distance=inf, k=1):                                  
            """Cross-match the values between X1 and X2                                 

            By default, this uses a KD Tree for speed.                                  

            Parameters                                                                  
            ----------                                                                  
            X1 : array_like                                                             
                first dataset, shape(N1, D)                                             
            X2 : array_like                                                             
                second dataset, shape(N2, D)                                            
            max_distance : float (optional)                                             
                maximum radius of search.  If no point is within the given radius,      
                then inf will be returned.                                              

            Returns                                                                     
            -------                                                                     
            dist, ind: ndarrays                                                         
                The distance and index of the closest point in X2 to each point in X1   
                Both arrays are length N1.                                              
                Locations with no match are indicated by                                
                dist[i] = inf, ind[i] = N2                                              
            """                                                                         
            X1 = asarray(X1, dtype=float)                                               
            X2 = asarray(X2, dtype=float)                                               
            N1, D = X1.shape                                                            
            N2, D2 = X2.shape                                                           
            if D != D2:                                                                 
                raise ValueError('Arrays must have the same second dimension')          
            kdt = cKDTree(X2)                                                           
            dist, ind = kdt.query(X1, k=k, distance_upper_bound=max_distance)           

            return dist, ind  

        def rexmatch(radec1, radec2, max_radius, k=1):
            """call crossmatch, remove multiple matched objects in list1 to one object in list 2
            Parameters
            ----------
            radec1 : array_like
                first dataset, shape(N1, D)
            radec2 : array_like
                second dataset, shape(N2, D)
            max_radius : float 
                maximum radius of search.  If no point is within the given radius,
                then inf will be returned.

            Returns
            -------
            match1, match2: int array (they have the same size)

            dist1 : float array    
                distance between matched star
            umatch1, umatch2: int array (not have the same size)    
                The index of un-matched star in radec1 and radec2
            """
            radec1 = radec1
            radec2 = radec2

            # crossmatch catalogs
            # max_radius = 1. / 3600  # 1 arcsec
            dist, ind = crossmatch(radec1, radec2, max_radius, k=k)

            # remove multiple objects in list1 to one object in list 2
            match1 = ~isinf(dist)
            sind = sorted(ind[match1])
            us, usidx = unique(sind, return_index=True)
            mi = len(radec2)
            for i in range(1,len(us)):
                diff = usidx[i]-usidx[i-1]
                if diff > 1 :
                    ri = (ind == us[i-1]).nonzero()
                    dist[ri] = inf
                    ind[ri] = mi

            match1 = ~isinf(dist)
            umatch1 = isinf(dist)

            inda = arange(len(radec2))
            umatch2 = list(set(inda)-set(ind))
            match2 = ind[match1]
            inda1 = arange(len(radec1))
            match1 = inda1[match1]
            umatch1 = inda1[umatch1]
            dist1 = dist[match1]

            return match1, match2, umatch1, umatch2, dist1

        class ptf_data:
            def __init__(self, fn1, fn2, coord, dm, sigma):
                rac, decc = coord
                if fn1.endswith('.cat'):
                    with open(fn1, 'r') as fnr1:
                        ptf1 = array([i.split() for i in fnr1.readlines()[14:]]).astype(float)
                    with open(fn2, 'r') as fnr2:
                        ptf2 = array([i.split() for i in fnr2.readlines()[14:]]).astype(float)
                    ra1, dec1 = ptf1[:,11], ptf1[:,12]
                    mag1, merr1 = ptf1[:,7], ptf1[:,8]
                    x1, y1 = ptf1[:,2], ptf1[:,3]
                    ra2, dec2 = ptf2[:,11], ptf2[:,12]
                    mag2, merr2 = ptf2[:,7], ptf2[:,8]
                    mrange = arange(7, 13, dm)
                elif fn1.endswith('.ctlg'):
                    with pyfits.open(fn1) as ptf1:
                        star1 = ptf1[1].data
                        idx = (star1['FLAGS'] & 506 == 0) * (star1['IMAFLAGS_ISO'] & 1821 == 0)
                        ra1   = star1['ALPHAWIN_J2000'][idx]
                        dec1  = star1['DELTAWIN_J2000'][idx]
                        mag1  = star1['MAG_AUTO'][idx]
                        merr1 = star1['MAGERR_AUTO'][idx]
                        x1, y1 = star1['XWIN_IMAGE'][idx],  star1['YWIN_IMAGE'][idx]
                        #x1, y1 = star1['XWIN_IMAGE'],  star1['YWIN_IMAGE']
                    with pyfits.open(fn2) as ptf2:
                        star2 = ptf2[1].data
                        idx = (star2['FLAGS'] & 506 == 0) * (star2['IMAFLAGS_ISO'] & 1821 == 0)
                        ra2   = star2['ALPHAWIN_J2000'][idx]
                        dec2  = star2['DELTAWIN_J2000'][idx]
                        mag2  = star2['MAG_AUTO'][idx]
                        merr2 = star2['MAGERR_AUTO'][idx]
                    mrange = arange(ceil(mag2.min())-1, ceil(mag2.max()), dm)


                ## match Ha656 and Ha663
                m1, m2, um1, um2, dist = rexmatch(zip(ra1,dec1), zip(ra2,dec2), 2./3600.)
                mdiff = mag2[m2]-mag1[m1]
                merr = sqrt(merr1[m1]**2.+merr2[m2]**2.)
                ram, decm, magm = ra2[m2], dec2[m2], mag2[m2]
                xm, ym = x1[m1], y1[m1]
                self.magm, self.mdiff = magm, mdiff

                mm1, mm2, umm1, umm2, distm = rexmatch(zip(mra,mdec), zip(ra1,dec1), 2./3600.)
                #print mra, mdec, ra1, dec1, ra2, dec2
                mmm1, mmm2, ummm1, ummm2, distmm = rexmatch(zip(mra,mdec), zip(ra2,dec2), 2./3600.)
                mdiffm = mag2[mmm2]-mag1[mm2]
                print mag2[mmm2], mag1[mm2], mdiffm, epic

                os.chdir('..')

                if len(mdiffm) != 0:   #若有HA663-HA656，才寫入csv檔並儲存

                    append_csv(epic, mag2[mmm2][0], mag1[mm2][0],  mdiffm[0])

                else :

                    append_csv(epic, 0, 0, 0)



                ## pick out outliers from mag difference
                self.ras, self.decs, self.xs, self.ys, self.ms, self.mds, self.mplt = [], [], [], [], [], [], []
                for i in mrange:
                    midx = (magm > i) * (magm < i+dm)
                    if midx.sum():
                        mdiff_tmp, magm_tmp, merr_tmp, ram_tmp, decm_tmp, x_tmp, y_tmp = \
                            mdiff[midx], magm[midx], merr[midx], ram[midx], decm[midx], xm[midx], ym[midx]
                        mdiff_clip = sta.sigma_clip(mdiff_tmp, sigma=3, iters=5)
                        if (~mdiff_clip.mask).sum() > 0:
                            wmeandiff = average(mdiff_clip.data[~mdiff_clip.mask], weights=merr_tmp[~mdiff_clip.mask])
                            stddiff = mdiff_clip.data[~mdiff_clip.mask].std()
                            photoerr = sqrt(1./sum(1./merr_tmp[~mdiff_clip.mask]**2.))
                            errsel = sqrt(photoerr**2.+stddiff**2.)
                            self.mplt.append([i, i+dm, wmeandiff, errsel])
                            sidx = ((mdiff_tmp-merr_tmp)-wmeandiff) > (sigma*errsel)
                            if sum(sidx) !=0 :
                                self.ras += list(ram_tmp[sidx])
                                self.decs += list(decm_tmp[sidx])
                                self.xs += list(x_tmp[sidx])
                                self.ys += list(y_tmp[sidx])
                                self.ms += list(magm_tmp[sidx])
                                self.mds += list(mdiff_tmp[sidx])
                        fig = pl.figure(num=0, figsize=(24,16), dpi = 100, facecolor='w', edgecolor='k')
                        #pl.plot(magm_tmp, mdiff_tmp, '.', color='gray', markersize=4)
                        pl.plot(magm_tmp, mdiff_tmp, '.', color='gray', markersize=4)
                        pl.plot(magm_tmp[sidx], mdiff_tmp[sidx], 'ro')
                        x = (i+(i+dm))/2
                        pl.errorbar(x, wmeandiff, yerr=errsel, fmt='o', markersize=5, mec='b', mfc='b', ecolor='b')
                        #pl.title("H663-H656 ="+str(mdiffm[0]))
                        #pl.plot(mag2[mmm2], mdiffm, 'g*', markersize=10)
                        #pl.text(mag2[mmm2]+0.01, mdiffm+0.01, ID)
                        pl.ylabel('H663-H656', size=20)
                        pl.xlabel('H663', size=20)
                        pl.xticks(size=15)
                        pl.yticks(size=15)

                #os.chdir("C:/Users/CareyJohn/Desktop/python_crawler/ktwo"+str(epic)+"_C1")
                #pl.savefig("C:/Users/CareyJohn/Desktop/python_crawler/ktwo"+str(epic)+"_C1/"+ID+"diagram.png" , format='png')
                #pl.show()
                #pl.clf()
        ptf_data(fn1, fn2, [mra,mdec], ptf_dm, ptf_sig)

        #os.chdir("D:/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1") #for windows
        os.chdir("/media/careyjohn/DATA/python_PTF_data_k2c1/ktwo"+str(epic)+"_C1") # for Linux

        #if not os.path.exists(ID+"diagram.png"):

        #pl.savefig('//media//careyjohn/OS//Users//CareyJohn//Desktop//PSROC_klpr//K2_PTF_ha//k2_camp1//'+ID+'//diagram.png' , format='png')
        #pl.savefig("C:/Users/CareyJohn/Desktop/python_crawler/ktwo"+str(epic)+"_C1/"+ID+"diagram.png" , format='png')
        os.chdir("..")

In [10]:
dfs = pd.read_csv('K2_M_C1_ptf_h.csv')

In [11]:
#dfs['EPIC'][0:2]
ra_list = list(dfs['RA'][0:8])
dec_list = list(dfs['DEC'][0:8])
epic_list = list(dfs['EPIC'][0:8])

In [12]:
epic_list
ra_list

[174.69463500000001,
 175.25185099999999,
 175.08766799999998,
 176.619427,
 176.61401000000001,
 174.53988100000001,
 175.24243799999999,
 174.48808]

In [16]:
for s in range(len(ra_list)):
    input_ra_dec(ra_list[s], dec_list[s], epic_list[s])

In [10]:
ra_list = list(dfs['RA'])
dec_list = list(dfs['DEC'])
epic_list = list(dfs['EPIC'])

In [11]:
epic_list
ra_list

[174.69463500000001,
 175.25185099999999,
 175.08766799999998,
 176.619427,
 176.61401000000001,
 174.53988100000001,
 175.24243799999999,
 174.48808,
 175.28499199999999,
 174.65675400000001,
 175.266256,
 175.221138,
 175.033714,
 173.90266,
 172.834407,
 174.74730400000001,
 173.73786699999999,
 173.160202,
 173.07843099999999,
 173.243607,
 174.853994,
 173.648977,
 175.0574,
 173.19734399999999,
 173.31251499999999,
 174.59721999999999,
 175.228656,
 172.68956599999999,
 172.962266,
 173.50260500000002,
 171.992041,
 174.54498000000001,
 174.74072699999999,
 173.74377799999999,
 172.55516699999998,
 174.01819599999999,
 171.82110800000001,
 175.10502600000001,
 174.858271,
 173.48012800000001,
 173.55076699999998,
 173.50273999999999,
 174.08188000000001,
 173.323511,
 170.99454399999999,
 175.193994,
 173.536235,
 171.13351,
 171.75766399999998,
 173.87864299999998,
 172.00163600000002,
 172.90977100000001,
 171.15782200000001,
 171.12313600000002,
 170.875834,
 172.7151389999999

In [12]:
#Running

for s in range(len(ra_list)):
    input_ra_dec(ra_list[s], dec_list[s], epic_list[s])

[-8.7295475] [-8.7973299] [ 0.0677824] 201767620
[-8.88867378] [-9.06109619] [ 0.17242241] 201767660
[-9.61551094] [-9.55767441] [-0.05783653] 201767870
[-7.33903646] [-7.7124939] [ 0.37345743] 201767895
[] [-5.88841343] [] 201768092
[-7.75177717] [-8.00637722] [ 0.25460005] 201768261
[] [] [] 201768649
[] [] [] 201768701
[] [] [] 201768702
[-8.27153969] [-8.44358158] [ 0.17204189] 201768722
[-9.46625137] [-9.51617432] [ 0.04992294] 201768912
[] [] [] 201768962
[-8.25352001] [-8.38997936] [ 0.13645935] 201769111
[-7.76886272] [-7.9513731] [ 0.18251038] 201769219
[-9.10190296] [-9.22051811] [ 0.11861515] 201769422
[-6.98268747] [-7.05336475] [ 0.07067728] 201769484
[-8.01805878] [-7.93926954] [-0.07878923] 201769811
[-8.1502409] [-8.06287384] [-0.08736706] 201769988
[-8.83099651] [-8.86900425] [ 0.03800774] 201770646
[-8.42630577] [-8.43478012] [ 0.00847435] 201770716
[-7.1397295] [-7.17643881] [ 0.03670931] 201770740
[-6.4546833] [-6.64068031] [ 0.18599701] 201770779
[-7.03428268] [-7.

In [43]:
#dfs2 = pd.read_csv('ptf_ha_substraction_k2c1.csv')

#EPIC2 = dfs2['EPIC']


def found_epic_wiht_no_ha663_sub_ha656(csv_name):
    
    dfs2 = pd.read_csv(str(csv_name))
    
    test_list = []
    
    EPIC2 = dfs2['EPIC']
    for i in range(len(EPIC2)):
        if dfs2['HA663-HA656'][i] == 0:
            test_list.append(EPIC2[i])
    return test_list



In [45]:
test_list2 = found_epic_wiht_no_ha663_sub_ha656('ptf_ha_substraction_k2c1.csv')

In [8]:
#test_list2

In [46]:
len(test_list2)

1166

In [47]:
3865-len(test_list2)

2699